In [1]:
import os
import glob
import pandas as pd
import numpy as np
import random
import collections
from vit_keras import vit, utils
import tensorflow_addons as tfa
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.metrics

In [ ]:
IMAGE_SIZE = 64

dataset_loc = '/home/ubuntu/Data/MedImage/'
image_folder = dataset_loc + "IDC_regular_ps50_idx5/"

MODEL_PARAMETERS = {"model_name" : "ViT_Train_3_weights",                    
                    "activation" : "relu", # can be relu / sigmoid - **need to google for binary classification,               
                    "loss" : "binary_crossentropy",
                    "optimizer" : "sgd",
                    "class_mode": "binary",
                    "output_layers" : 1,
                    "output_activation" : "sigmoid",
                    "num_epochs" : 50,
                    "batch_size" : 128}

model_output_folder = dataset_loc + MODEL_PARAMETERS["model_name"] + "/"